# covariance

Understanding how `np.correlate` can lead to auto and cross-correlations

For discussion about biased and unbiased autocovariance estimate statistical properties see Priestley chapter 5 as well as notebook `autocovariance.ipynb`.

In [1]:
import numpy as np

Compute the weights associated with mode `full`

In [2]:
a = np.array([1,1,1,1])
b = np.array([1,1,1,1])
np.correlate(a, b, mode="full")

array([1, 2, 3, 4, 3, 2, 1])

Figure out which timeseries is lagged compared to the other one

In [3]:
b = np.array([1,1,.1,.1])
np.correlate(a, b, mode="full")

array([0.1, 0.2, 1.2, 2.2, 2.1, 2. , 1. ])

The autocorrelation thus indicates the second timeseries (`b`) is shifted:

$c(n-1+i) = \sum_j a(j) b(j+i)$

Comparison with mode `same`and `valid`

In [4]:
a = np.array([1,1,1,1])
b = np.array([1,1,.1,.1])
np.correlate(a, b, mode="same")

array([0.2, 1.2, 2.2, 2.1])

In [5]:
np.correlate(a, b, mode="valid")

array([2.2])

---

## build wrapper around np.correlate


In [16]:
def correlate(u, v, 
              biased=True,
              one_sided=True,
              weights=False,
             ):
    """ custom correlation
    
      corr[lag] = 1/w(lag) sum_lag u(t) x v(t+lag)
      
    
    Parameters
    ----------
    u, v: np.array
        Input timeseries, must be of the same length
    biased: boolean, optional
        Returns a biased estimation of the correlation. Default is True
        Biased: corr[lag] = 1/N sum ...
        Unbiased: corr[lag] = 1/(N-lag) sum ...
    one_sided: boolean, optional
        Outputs only positive lag. Default is True
    weights: boolean, optional
        Returns weights. Default is False
    
    Returns
    -------
    c: np.array
        Autocorrelation
    lag: np.array of int
        Lag in index (nondimensional units)
    w: np.array of int
        Weights used for the calculation of the autocorrelation
        
    """
    n = u.size
    assert u.size==v.size, "input vectors must have the same size"
    # build arrays of weights
    if biased:
        w = n
    else:
        _w = np.arange(1,n+1)
        w = np.hstack([_w, _w[-2::-1]])
    #
    c = np.correlate(u, v, mode="full") / w
    lag = np.arange(-n+1,n)
    #
    if one_sided:
        c, lag = c[n-1:], lag[n-1:]
        if not biased:
            w = w[n-1:]
    if weights:
        return c, lag, w
    else:
        return c, lag

Check that the result is consistent with expectations

**Unbiased estimate:**

In [18]:
a = np.array([1,1,1,1])
b = np.array([1,1,1,1])
correlate(a,b, biased=False, weights=True)

(array([1., 1., 1., 1.]), array([0, 1, 2, 3]), array([4, 3, 2, 1]))

In [19]:
a = np.array([1,.1,.1,1])
b = np.array([1,1,.1,.1])
correlate(a,b, biased=False, weights=True)

(array([0.3025, 0.1   , 0.55  , 1.    ]),
 array([0, 1, 2, 3]),
 array([4, 3, 2, 1]))

In [20]:
a = np.array([1,1,1,1])
b = np.array([1,1,1,1])
correlate(a,b, one_sided=False, biased=False, weights=True)

(array([1., 1., 1., 1., 1., 1., 1.]),
 array([-3, -2, -1,  0,  1,  2,  3]),
 array([1, 2, 3, 4, 3, 2, 1]))

In [21]:
a = np.array([1,1,1,1])
b = np.array([1,1,.1,.1])
correlate(a,b, one_sided=False, biased=False, weights=True)

(array([0.1 , 0.1 , 0.4 , 0.55, 0.7 , 1.  , 1.  ]),
 array([-3, -2, -1,  0,  1,  2,  3]),
 array([1, 2, 3, 4, 3, 2, 1]))

**Biased estimate:**

In [24]:
a = np.array([1,1,1,1])
b = np.array([1,1,.1,.1])
correlate(a,b, one_sided=False, biased=True, weights=True)

(array([0.025, 0.05 , 0.3  , 0.55 , 0.525, 0.5  , 0.25 ]),
 array([-3, -2, -1,  0,  1,  2,  3]),
 4)